In [11]:
import numpy as np
import google.auth
from google.cloud import bigquery


credentials, project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

bq_client = bigquery.Client(credentials=credentials, project=project_id,)




In [13]:
dataset_id="model_space"
model_space
try:
    model_space = bq_client.get_dataset(dataset_id)  
    print("Dataset {} already exists".format(dataset_id))
except NotFound:
    print("Dataset {} is not found".format(dataset_id))
    model_space = bq_client.create_dataset(dataset_id) 

Dataset model_space already exists


In [14]:
%%bigquery
CREATE OR REPLACE MODEL model_space.arrdelay
OPTIONS
  (model_type='linear_reg', input_label_cols=['avg_delay_dur']) AS
SELECT
  avg_delay_dur,
  airline,
  departure_airport,
  arrival_airport,
  departure_schedule,
FROM
  `red-beryl-labs.redbq.flight_records`
WHERE
  avg_delay_dur IS NOT NULL


Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1077.30query/s]                        


""


In [ ]:
%%bigquery
SELECT * FROM ml.PREDICT(MODEL model_space.arrdelay, (
SELECT 
  'AA' as airline,
  'DFW' as departure_airport,
  'LAX' as arrival_airport,
  dep_delay,
  18 as taxi_out,
  1235 as distance
FROM
  UNNEST(GENERATE_ARRAY(-3, 10)) as dep_delay
))